In [9]:
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import date, timedelta
import csv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
import string
import nltk

Будем парсить сайт https://ria.ru - зададим стартовые URL, с которых начнется поиск, также зададим, что будут участвовать в поиске новости, опубликованные в течении прошлых 10 дней. Сначала с помощью функции get_parsing_urls() получим спсиок неповтоярющихся URL, затем с каждого URL функцией get_article_from_url() получим текст новости, количество просмотров, лайков, дизлайков и дату публикации: 

In [10]:
start_dt = date.today()
end_dt = start_dt + timedelta(-10)
start_dt_url = start_dt.strftime('%Y%m%d')
date_url_list = []

for n in range(int((start_dt - end_dt).days) + 1):
    date_url_list.append((end_dt + timedelta(n)).strftime('%Y%m%d'))


def get_parsing_urls():
    url_more_list = []
    for date_url in date_url_list:
        url = f'https://ria.ru/services/lenta/more.html?id=1821313755&date={date_url}T000000&onedayonly=1&articlemask=lenta_common'
        url_more_list.append(url)

    url_list = ['https://ria.ru/',
                'https://ria.ru/politics/',
                'https://ria.ru/society/',
                'https://ria.ru/economy/',
                'https://ria.ru/world/',
                'https://ria.ru/incidents/',
                'https://ria.ru/science/',
                'https://ria.ru/culture/',
                'https://ria.ru/religion/',
                'https://ria.ru/tourism/',
                'https://ria.ru/export/rss2/archive/index.xml',
                'https://ria.ru/archive/']

    url_list.extend(url_more_list)
    url_start_list = url_list.copy()
    url_set = set()
    url_set.update(url_list)

    for url in url_list:
        counter = 0
        response = requests.get(url)
        time.sleep(0.3)
        result = re.finditer(r'(https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]).html',
                             response.text)
        for grp in result:
            if grp.group(0).find('ria.ru') > -1:
                url_set.add(grp.group(0))
                url_list.append(grp.group(0))
                counter += 1
        url_list = list(set(url_list))
        print(f'Обработано: {counter} URL')

    for start_url in url_start_list:
        url_set.discard(start_url)

    print(f'Длина списка URL: {len(url_set)}')
    return url_set


def get_article_from_url(parse_url):
    result_dict = {}

    response = requests.get(parse_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        article_body = soup.find_all('div', class_="article__text")
        article_text = ''

        for divs in article_body:
            temp_str = re.sub(r'(<a[a-zA-Z\d\-\.\:\@\?\=\_\/\"\s]*\">)', '', str(divs), count=0, flags=0)    #\"_blank
            temp_str = re.sub(r'<strong.*strong>', '', temp_str, count=0, flags=0)
            clean_str = temp_str.replace('<div class="article__text">', '').replace('</div>', '').replace('</a>', '').replace('  ', ' ')
            article_text += clean_str + ' '

        article_id = soup.find('div', class_='share m-header')['data-id']
        article_date = soup.find('div', class_='article__info-date').find('a').contents[0]
    except (TypeError, AttributeError):
        return -1

    metrics_url = f'https://ria.ru/services/dynamics/{start_dt_url}/{article_id}.html'

    response = requests.get(metrics_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article_seen = 0
    article_likes = 0
    article_dislikes = 0
    try:
        article_seen = soup.find('span', class_='statistic__item m-views').contents[1]
        article_likes = soup.find('a', class_='emoji-item m-type-s1 m-active').find('span', class_='m-value').contents[0]
        article_dislikes = soup.find('a', class_='emoji-item m-type-s6 m-active').find('span', class_='m-value').contents[0]
    except AttributeError:
        pass

    result_dict['article_text'] = article_text
    result_dict['article_seen'] = article_seen
    result_dict['article_likes'] = article_likes
    result_dict['article_dislikes'] = article_dislikes
    result_dict['article_date'] = article_date
    return result_dict if len(article_text) > 0 else -1

Сохраним все данные в CSV файл для дальнейшей работы:

In [11]:
with open('ria_news.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    urls_parsing_list = list(get_parsing_urls())
    urls_parsing_count = len(urls_parsing_list)
    for index, parsing_url in enumerate(urls_parsing_list):
        print(f'Обрабатываем {round((index + 1) * 100.0 / urls_parsing_count, 2)}% URL: {parsing_url}')
        result = get_article_from_url(parsing_url)
        if isinstance(result, dict):
            writer.writerow(result.values())

Обработано: 106 URL
Обработано: 64 URL
Обработано: 40 URL
Обработано: 40 URL
Обработано: 64 URL
Обработано: 64 URL
Обработано: 126 URL
Обработано: 202 URL
Обработано: 155 URL
Обработано: 168 URL
Обработано: 200 URL
Обработано: 64 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 60 URL
Обработано: 33 URL
Обработано: 33 URL
Обработано: 36 URL
Обработано: 36 URL
Обработано: 27 URL
Обработано: 27 URL
Обработано: 33 URL
Обработано: 33 URL
Обработано: 32 URL
Обработано: 32 URL
Обработано: 33 URL
Обработано: 33 URL
Обработано: 33 URL
Обработано: 33 URL
Обработано: 33 URL
Обработано: 33 URL
Обработано: 29 URL
Обработано: 29 URL
Обработано: 27 URL
Обработано: 27 URL
Обработано: 30 URL
Обработано: 30 URL
Обработано: 41 URL
Обработано: 41 URL
Обработано: 41 URL
Обработано: 41 URL
Обработано: 41 URL
Обработано: 41 URL
Обработано: 33 URL
Обрабо

Обработано 773 URL, результат сохранен в файл ria_news.csv, будем работать с ним.

In [9]:
df = pd.read_csv('ria_news.csv', sep=',')

In [10]:
df.shape

(664, 5)

Итого имеем данные из 663 новостей 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   article_text      664 non-null    object
 1   article_seen      664 non-null    int64 
 2   article_likes     664 non-null    int64 
 3   article_dislikes  664 non-null    int64 
 4   article_date      664 non-null    object
dtypes: int64(3), object(2)
memory usage: 26.1+ KB


In [12]:
df.isnull().sum()

article_text        0
article_seen        0
article_likes       0
article_dislikes    0
article_date        0
dtype: int64

Пропусков в данных нет - они были отброшены парсером.

In [13]:
df.head()

,article_text,article_seen,article_likes,article_dislikes,article_date
0,Ученые ВятГУ изучили влияние тренировочного пр...,1031,18,0,09:00 04.10.2022
1,Белый дом отказался комментировать получение ...,18893,19,1,22:43 26.09.2022
2,Мусульмане отмечают одно из главных событий ис...,2447,47,63,14:16 31.07.2020
3,"В четверг стало известно, что в пятницу 30 се...",6679,39,9,23:21 29.09.2022
4,США не исключают возможности поставок Украине...,26438,51,31,22:51 30.09.2022


Загружаем список стоп-слов для русского языка:

In [16]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rpure\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [17]:
stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на